In [1]:
import sys
sys.path.append("../")
print(sys.path)

['/home/varun/EnvisEdge/notebooks', '/home/varun/EnvisEdge/notebooks', '/usr/lib/python38.zip', '/usr/lib/python3.8', '/usr/lib/python3.8/lib-dynload', '', '/home/varun/env/envis/lib/python3.8/site-packages', '/home/varun/.local/lib/python3.8/site-packages', '../']


In [ ]:
import yaml
import json
from json import dumps, loads
from kafka import KafkaProducer, KafkaConsumer
from fedrec.data_models.job_submit_model import JobSubmitMessage
from fedrec.utilities import registry
import fedrec
import experiments
import fl_strategies
import datasets
with open("../configs/regression.yml", 'r') as cfg:
    config = yaml.load(cfg, Loader=yaml.FullLoader)

def init_kafka(config):
    producer_url = "{}:{}".format(
        config["producer_url"], config["producer_port"])
    return KafkaProducer(
        bootstrap_servers=[producer_url])

# serializer = registry.construct("serializer", "json")
# config = config["multiprocessing"]["communications"]

producer = init_kafka(config["multiprocessing"]["communication_interface"])
producer.send('job-request-aggregator', value=str({"job_type":"train", "results": {},"error": [1,2]}).encode('utf-8'))


In [ ]:
def init_kafka(config):
    consumer_url = "{}:{}".format(
        config["consumer_url"], config["consumer_port"])
    return KafkaConsumer(
        bootstrap_servers=[consumer_url])



In [20]:
from fedrec.python_executors.aggregator import Aggregator, Neighbour
from fedrec.utilities.logger import NoOpLogger
from fedrec.data_models.state_tensors_model import StateTensors
from inspect import ismethod, getmembers, isfunction
from fedrec.utilities import registry
import torch
import yaml 

import fedrec
import experiments
import fl_strategies
import datasets

with open("../configs/regression.yml", 'r') as cfg:
    config = yaml.load(cfg, Loader=yaml.FullLoader)

ag_config = {
        # Seed for RNG used in shuffling the training data.
    "data_seed" : 100,
    # Seed for RNG used in initializing the model.
    "init_seed" : 100,
    # Seed for RNG used in computing the model's training loss.
    # Only relevant with internal randomness in the model, e.g. with dropout.
    "model_seed" : 100
}
tensor = StateTensors(
        storage='/home/varun/dump_tensor/',
        worker_id=0, round_idx=0,
        tensors=torch.load(
            '/home/varun/dump_tensor/worker_id_0/0_0_trainer45.pt'),
        tensor_type='trainer',
        suffix="41")

in_neighbours={
            0: Neighbour(0, tensor, 5)}
worker=registry.construct('aggregator',
                        config['aggregator'],
                        unused_keys=(),
                        config_dict=config,
                        in_neighbours=in_neighbours,
                        out_neighbours={})
print({
    func_name: getattr(worker, func_name)
    for func_name in dir(worker)
    if callable(getattr(worker, func_name))
})

print("*****************************")

print(
    {
    func_name_list[0]: getattr(worker, func_name_list[0])
    for func_name_list in getmembers(worker, predicate=ismethod)
}
)

{'__class__': <class 'fl_strategies.fed_avg.FedAvg'>, '__delattr__': <method-wrapper '__delattr__' of FedAvg object at 0x7f23d8b31a90>, '__dir__': <built-in method __dir__ of FedAvg object at 0x7f23d8b31a90>, '__eq__': <method-wrapper '__eq__' of FedAvg object at 0x7f23d8b31a90>, '__format__': <built-in method __format__ of FedAvg object at 0x7f23d8b31a90>, '__ge__': <method-wrapper '__ge__' of FedAvg object at 0x7f23d8b31a90>, '__getattribute__': <method-wrapper '__getattribute__' of FedAvg object at 0x7f23d8b31a90>, '__gt__': <method-wrapper '__gt__' of FedAvg object at 0x7f23d8b31a90>, '__hash__': <method-wrapper '__hash__' of FedAvg object at 0x7f23d8b31a90>, '__init__': <bound method FedAvg.__init__ of <fl_strategies.fed_avg.FedAvg object at 0x7f23d8b31a90>>, '__init_subclass__': <built-in method __init_subclass__ of type object at 0x7eb9c40>, '__le__': <method-wrapper '__le__' of FedAvg object at 0x7f23d8b31a90>, '__lt__': <method-wrapper '__lt__' of FedAvg object at 0x7f23d8b31a

In [18]:
getmembers(worker, predicate=ismethod)

[('__init__',
  <bound method FedAvg.__init__ of <fl_strategies.fed_avg.FedAvg object at 0x7f23d8501a60>>),
 ('_get_default_state',
  <bound method EnvisBase._get_default_state of <fl_strategies.fed_avg.FedAvg object at 0x7f23d8501a60>>),
 ('_set_state',
  <bound method EnvisBase._set_state of <fl_strategies.fed_avg.FedAvg object at 0x7f23d8501a60>>),
 ('aggregate',
  <bound method FedAvg.aggregate of <fl_strategies.fed_avg.FedAvg object at 0x7f23d8501a60>>),
 ('sample_clients',
  <bound method FedAvg.sample_clients of <fl_strategies.fed_avg.FedAvg object at 0x7f23d8501a60>>),
 ('store_state',
  <bound method FedAvg.store_state of <fl_strategies.fed_avg.FedAvg object at 0x7f23d8501a60>>),
 ('update',
  <bound method EnvisBase.update of <fl_strategies.fed_avg.FedAvg object at 0x7f23d8501a60>>)]